In [1]:
import tensorflow as tf

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default
#tf.executing_eagerly()
tf.compat.v1.enable_eager_execution(
    config=None,
    device_policy=None,
    execution_mode=None
)

In [3]:
tf.__version__

'1.15.0'

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [5]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/gdrive/My Drive/Colab Notebooks/Residency6/Project-R6(NN)/bank.csv')

In [8]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
data.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [10]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
data = pd.get_dummies(data,columns=['Gender'])

In [0]:
data = pd.get_dummies(data,columns=['Geography'])

In [13]:
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,1,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,1,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,1,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0,0,0,1


In [14]:
data.shape

(10000, 14)

In [15]:
data.isna().sum()

CreditScore          0
Age                  0
Tenure               0
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
Gender_Female        0
Gender_Male          0
Geography_France     0
Geography_Germany    0
Geography_Spain      0
dtype: int64

In [0]:
X = data[['CreditScore','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','EstimatedSalary','Gender_Female','Gender_Male','Geography_France','Geography_Germany','Geography_Spain']]
y = data[['Exited']]

In [0]:
from sklearn.model_selection import train_test_split
test_size = 300 # taking 300 & 95 as training and test set sizes
seed = 7  # Random numbmer seeding for reapeatability of the code
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=test_size, random_state=seed)

In [0]:
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')
X_test = X_test.astype('float32')
y_test = y_test.astype('float32')

In [0]:
import numpy as np

In [0]:
y_train = np.array(y_train)

In [0]:
y_test = np.array(y_test)

In [0]:
from sklearn.preprocessing import Normalizer

In [0]:
transformer = Normalizer()
X_train = transformer.fit_transform(X_train)

In [0]:
transformer = Normalizer()
X_test = transformer.fit_transform(X_test)

In [0]:
model = tf.keras.models.Sequential()

In [0]:
model.add(tf.keras.layers.Dense(2, input_shape = (13,), activation='relu'))

In [0]:
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [0]:
model.compile(optimizer='adam', loss='binary_crossentropy', 
              metrics=['accuracy'])

In [29]:
model.fit(X_train,y_train,          
          validation_data=(X_test,y_test),
          epochs=100,
          batch_size=32) 

Train on 9700 samples, validate on 300 samples
Epoch 1/100
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
9700/9700 [==============================] - 2s 211us/sample - loss: 0.6325 - acc: 0.7095 - val_loss: 0.5351 - val_acc: 0.8133
Epoch 2/100
9700/9700 [==============================] - 1s 56us/sample - loss: 0.5198 - acc: 0.7958 - val_loss: 0.4784 - val_acc: 0.8133
Epoch 3/100
9700/9700 [==============================] - 1s 58us/sample - loss: 0.5041 - acc: 0.7958 - val_loss: 0.4738 - val_acc: 0.8133
Epoch 4/100
9700/9700 [==============================] - 1s 54us/sample - loss: 0.5033 - acc: 0.7958 - val_loss: 0.4729 - val_acc: 0.8133
Epoch 5/100
9700/9700 [==============================] - 1s 58us/sample - loss: 0.5030 - acc: 0.7958 - val_loss: 0.4727 - val_acc: 0.8133
Epoch 6/100
9700/9700 [==============================] - 1s 57us/sample - loss: 0.5028 - acc: 0.7958 - val_loss: 0.4715 - val_acc: 0.8133
Epoch 7/100
9700/9700 [=======

In [0]:
#input = np.array(X_test[0:3])

In [0]:
#pred = np.argmax(model.predict(input))
#pred

In [32]:
# predict probabilities for test set
yhat_probs = model.predict(X_test, verbose=0)
# predict crisp classes for test set
yhat_classes = model.predict_classes(X_test, verbose=0)
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]

# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes)
print('Accuracy: %f' % accuracy)

# confusion matrix
matrix = confusion_matrix(y_test, yhat_classes)
print(matrix)

Accuracy: 0.813333
[[244   0]
 [ 56   0]]
